# **Connect to drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Install dependencies**

In [ ]:
!pip install simpletransformers
!pip install nltk
!pip install wandb
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 21.3 MB/s eta 0:00:00
   ━

# **Import dataset**

In [ ]:
import json
with open(r"dataset\train_dataset.json", "r") as train_dataset:
      train = json.load(train_dataset)

In [ ]:
train

[{'context': 'Computational complexity theory is a branch of the theory of computation in theoretical computer science that focuses on classifying computational problems according to their inherent difficulty, and relating those classes to each other. A computational problem is understood to be a task that is in principle amenable to being solved by a computer, which is equivalent to stating that the problem may be solved by mechanical application of mathematical steps, such as an algorithm.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'What branch of theoretical computer science deals with broadly classifying computational problems by difficulty and class of relationship?',
    'answers': [{'text': 'Computational complexity theory',
      'answer_start': 0}]},
   {'id': '00002',
    'is_impossible': False,
    'question': 'By what main attribute are computational problems classified utilizing computational complexity theory?',
    'answers': [{'text': 'inher

In [ ]:
with open(r"dataset\test_dataset.json", "r") as test_dataset:
      test = json.load(test_dataset)

In [ ]:
test

[{'context': 'The best, worst and average case complexity refer to three different ways of measuring the time complexity (or any other complexity measure) of different inputs of the same size.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'What are the three primary expressions used to represent case complexity?',
    'answers': [{'text': 'best, worst and average', 'answer_start': 4}]}]},
 {'context': 'Upper and lower bounds are usually stated using the big O notation, which hides constant factors and smaller terms.',
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'What expression is generally used to convey upper or lower bounds?',
    'answers': [{'text': 'big O notation', 'answer_start': 52}]},
   {'id': '00003',
    'is_impossible': True,
    'question': 'What does a big O notation hide?',
    'answers': [{'text': 'constant factors and smaller terms',
      'answer_start': 80}]}]},
 {'context': 'Many important complexity classes can 

In [ ]:
with open(r"dataset\prediction_dataset.json", "r") as prediction_dataset:
      prediction = json.load(prediction_dataset)

In [ ]:
prediction

[{'context': "Lionel Messi is an Argentine professional footballer widely considered one of the greatest players of all time. He has spent the majority of his career at Barcelona, where he has won numerous trophies including multiple UEFA Champions League titles and Ballon d'Or awards. Messi is known for his incredible dribbling, precise finishing, and vision on the field.",
  'qas': [{'id': '001',
    'question': 'Which football club has Lionel Messi spent most of his career at?'},
   {'id': '002',
    'question': 'What is Lionel Messi widely considered to be in the world of football?'},
   {'id': '003',
    'question': "What are some of Lionel Messi's known skills on the football field?"}]},
 {'context': 'Serena Williams is an American professional tennis player who is regarded as one of the greatest female tennis players of all time. With multiple Grand Slam titles to her name, including Wimbledon, the US Open, and the Australian Open, Williams has cemented her legacy in the sport. 

# **Importing Library**

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
import os
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
import wandb

In [ ]:
# Download punctuation
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# # Example if using Tokenization for cleaning Dataset

# # Clean text
# def clean_text(text):
#     text = text.encode('ascii', 'ignore').decode()
#     text = re.sub(r'[^a-zA-Z0-9\s\.,\?!]', '', text)
#     text = re.sub(r'\s+', ' ', text)
#     return text

# # Tokenization
# def tokenize_text(text):
#     tokens = word_tokenize(text)
#     return tokens

# # Token into sentences
# def join_tokens(tokens):
#     sentence = ' '.join(tokens)
#     return sentence

# # Combining tokens into sentences in each part
# for item in prediction:
#     item['context'] = join_tokens(tokenize_text(clean_text(item['context'])))
#     for qa in item['qas']:
#         qa['question'] = join_tokens(tokenize_text(clean_text(qa['question'])))

# # Create new clean dataset
# with open('/content/drive/MyDrive/ColabNotebooks/cleaned_tokenized_and_joined_prediction_dataset.json', 'w') as file:
#     json.dump(prediction, file, indent=4)

# **Importing library and set up model Question Answering using simpletransformer and roberta pretrained model**

In [ ]:
# define the model name
model_type="roberta"
model_name= "roberta-base"

In [ ]:
# configure the model and setting hyperparameter
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 64
model_args.evaluate_during_training=True
model_args.early_stopping_patience = 3
model_args.n_best_size=1
model_args.num_train_epochs=100
model_args.eval_batch_size=64
model_args.early_stopping_delta = 0
model_args.overwrite_output_dir=True
model_args.use_cached_eval_features=True
model_args.output_dir=f"output/{model_type}"
model_args.best_model_dir=f"best_model"
model_args.max_seq_length=256
model_args.evaluate_during_training_steps=1000
model_args.save_model_every_epoch=False
model_args.save_eval_checkpoints=False
model_args.reprocess_input_data=True
model_args.early_stopping_metric_minimize = False
model_args.use_cuda=True
model_args.use_wandb=True
model_args.wandb_project="QuestionAnswering"
model_args.wandb_kwargs={"name": model_name}

# **Set up api key wandb**

In [ ]:
# input wandb api key for visualization
wandb_api_key = os.getenv("WANDB_API_KEY")
wandb.login(key=wandb_api_key)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Hyperparameter default from QuestionAnsweringArgs simpletransformer library
#     adafactor_beta1: float = None,
#     adafactor_clip_threshold: float = 1,
#     adafactor_decay_rate: float = -0.8,
#     adafactor_eps: tuple = lambda : (1e-30, 0.001),
#     adafactor_relative_step: bool = True,
#     adafactor_scale_parameter: bool = True,
#     adafactor_warmup_init: bool = True,
#     adam_betas: tuple = lambda : (0.9, 0.999),
#     adam_epsilon: float = 1e-8,
#     best_model_dir: str = "outputs/best_model",
#     cache_dir: str = "cache_dir/",
#     config: dict = dict,
#     cosine_schedule_num_cycles: float = 0.5,
#     custom_layer_parameters: list = list,
#     custom_parameter_groups: list = list,
#     dataloader_num_workers: int = 0,
#     do_lower_case: bool = False,
#     dynamic_quantize: bool = False,
#     early_stopping_consider_epochs: bool = False,
#     early_stopping_delta: float = 0,
#     early_stopping_metric: str = "correct",
#     early_stopping_metric_minimize: bool = False,
#     early_stopping_patience: int = 3,
#     encoding: str = None,
#     eval_batch_size: int = 100,
#     evaluate_during_training: bool = False,
#     evaluate_during_training_silent: bool = True,
#     evaluate_during_training_steps: int = 2000,
#     evaluate_during_training_verbose: bool = False,
#     evaluate_each_epoch: bool = True,
#     fp16: bool = True,
#     gradient_accumulation_steps: int = 1,
#     learning_rate: float = 0.00004,
#     local_rank: int = -1,
#     logging_steps: int = 50,
#     loss_type: str = None,
#     loss_args: dict = dict,
#     manual_seed: int = None,
#     max_grad_norm: float = 1,
#     max_seq_length: int = 128,
#     model_name: str = None,
#     model_type: str = None,
#     multiprocessing_chunksize: int = -1,
#     n_gpu: int = 1,
#     no_cache: bool = False,
#     no_save: bool = False,
#     not_saved_args: list = list,
#     num_train_epochs: int = 1,
#     optimizer: str = "AdamW",
#     output_dir: str = "outputs/",
#     overwrite_output_dir: bool = False,
#     polynomial_decay_schedule_lr_end: float = 1e-7,
#     polynomial_decay_schedule_power: float = 1,
#     process_count: int = get_default_process_count,
#     quantized_model: bool = False,
#     reprocess_input_data: bool = True,
#     save_best_model: bool = True,
#     save_eval_checkpoints: bool = True,
#     save_model_every_epoch: bool = True,
#     save_optimizer_and_scheduler: bool = True,
#     save_steps: int = 2000,
#     scheduler: str = "linear_schedule_with_warmup",
#     silent: bool = False,
#     skip_special_tokens: bool = True,
#     tensorboard_dir: str = None,
#     thread_count: int = None,
#     tokenizer_name: str = None,
#     tokenizer_type: str = None,
#     train_batch_size: int = 8,
#     train_custom_parameters_only: bool = False,
#     trust_remote_code: bool = False,
#     use_cached_eval_features: bool = False,
#     use_early_stopping: bool = False,
#     use_hf_datasets: bool = False,
#     use_multiprocessing: bool = True,
#     use_multiprocessing_for_evaluation: bool = True,
#     wandb_kwargs: dict = dict,
#     wandb_project: str = None,
#     warmup_ratio: float = 0.06,
#     warmup_steps: int = 0,
#     weight_decay: float = 0,
#     model_class: str = "QuestionAnsweringModel",
#     doc_stride: int = 384,
#     lazy_loading: bool = False,
#     max_answer_length: int = 100,
#     max_query_length: int = 64,
#     n_best_size: int = 20,
#     null_score_diff_threshold: float = 0,
#     special_tokens_list: list = list

In [ ]:
# define the model
model = QuestionAnsweringModel(
      model_type,
      model_name,
      args=model_args,
      use_cuda=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# train the model with train dataset and evaluation with test dataset
model.train_model(train, eval_data=test)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
add example index and unique id: 100%|██████████| 50/50 [00:00<00:00, 307500.29it/s]


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

wandb: Currently logged in as: zulfikarahmad12. Use `wandb login --relogin` to force relogin


Running Epoch 1 of 100:   0%|          | 0/1 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 18/18 [00:00<00:00, 379.70it/s]

add example index and unique id: 100%|██████████| 18/18 [00:00<00:00, 67049.26it/s]


...

Running Epoch 99 of 100:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 100 of 100:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(100,
 {'global_step': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100],
  'correct': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,

In [ ]:
# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Check the Question Answer System andMake predictions with the model
question_test = [
      {
        "context": "Lionel Messi is an Argentine professional footballer widely considered one of the greatest players of all time. He has spent the majority of his career at Barcelona, where he has won numerous trophies including multiple UEFA Champions League titles and Ballon d'Or awards. Messi is known for his incredible dribbling, precise finishing, and vision on the field.",
        "qas": [
            {
                "id": "001",
                "question": "Which football club has Lionel Messi spent most of his career at?"
            }
          ]
      }
  ]

In [ ]:
answers, probabilities = model.predict(question_test)

print(answers)
print(probabilities)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 4006.02it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '001', 'answer': ['Barcelona,']}]
[{'id': '001', 'probability': [0.9999999998989721]}]


In [ ]:
import shutil

# Compress the model directory into a zip file
shutil.make_archive("best_model", 'zip', "best_model")

'/content/best_model.zip'